In [9]:
from sklearn.datasets import fetch_openml

In [25]:
mnist = fetch_openml("mnist_784", version=1, as_frame=True)
X, y = mnist["data"].to_numpy(), mnist["target"].to_numpy()
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
y_train = y_train.astype('int')
y_test = y_test.astype('int')

# Feature engineering

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))

# Modeling

In [27]:
from tensorflow import keras 

model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu", input_shape=[28*28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])
history = model.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4140 - accuracy: 0.8860
Epoch 2/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1801 - accuracy: 0.9482
Epoch 3/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1328 - accuracy: 0.9618
Epoch 4/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1048 - accuracy: 0.9699
Epoch 5/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0863 - accuracy: 0.9749
Epoch 6/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0720 - accuracy: 0.9797
Epoch 7/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0604 - accuracy: 0.9834
Epoch 8/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0516 - accuracy: 0.9863
Epoch 9/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0443 - accuracy: 0.9883
Epoch 10/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.038

In [28]:
y_test = y_test.astype('int')
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.1184 - accuracy: 0.9726


[0.11842259764671326, 0.972599983215332]

<hr>

# BACKUP (in case of low accuracy)

# Data augmentation

## Shift

In [ ]:
from scipy.ndimage.interpolation import shift

In [ ]:
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

In [ ]:
X_train_small_augmented = X_train_small.tolist()
y_train_small_augmented = y_train_small.tolist()

In [ ]:
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train_small, y_train_small):
        X_train_small_augmented.append(shift_image(image, dx, dy))
        y_train_small_augmented.append(label)

X_train_small_augmented = np.array(X_train_small_augmented)
y_train_small_augmented = np.array(y_train_small_augmented)

In [ ]:
shuffle_idx = np.random.permutation(len(X_train_small_augmented))
X_train_small_augmented = X_train_small_augmented[shuffle_idx]
y_train_small_augmented = y_train_small_augmented[shuffle_idx]

## Rotate

In [ ]:
from scipy.ndimage.interpolation import rotate

def rotate_img(image, ang = 5):
    '''
    Rotates passed images by given angle.
    '''
    image = image.reshape((28, 28))
    rotated_image = rotate(image, ang, reshape=False)
    return np.array(rotated_image.reshape([-1]))

In [ ]:
def array_for(x):
    '''
    Applies rotation to entire passed array.
    '''
    return np.array([rotate_img(xi) for xi in x])

In [ ]:
# rotate every image in the original dataset
X_train_small_rotated = array_for(X_train_small)
X_train_small_rotated.shape

In [ ]:
# add rotated data to the original one
y_train_small_double = np.concatenate([y_train_small, y_train_small])
X_train_small_rotated_double = np.concatenate((X_train_small_rotated, X_train_small), axis=0)